<a href="https://colab.research.google.com/github/luram94/OpenFOAM-4.x/blob/master/content_generation_pipeline_muscular.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install vllm
!pip install langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.1/131.1 MB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.2/307.2 kB 18.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 324.1/324.1 kB 27.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 53.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.9/42.9 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.7/65.7 MB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 222.7/222.7 MB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.1/37.1 MB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.7/111.7 kB 15.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━

# Funciones

In [ ]:
def clean_raw_text(raw_text):
    while "••" in raw_text:
        raw_text = raw_text.replace("••","•")

    while "  " in raw_text:
        raw_text = raw_text.replace("  "," ")
    while "...." in raw_text:
        raw_text = raw_text.replace("....","...")

    while "''''" in raw_text:
        raw_text = raw_text.replace("''''","'''")

    while "\n \n \n" in raw_text:
        raw_text = raw_text.replace("\n \n \n","\n \n")

    while "\n\n\n" in raw_text:
        raw_text = raw_text.replace("\n\n\n","\n\n")

    return raw_text

In [ ]:
from vllm import LLM, SamplingParams
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Load Model

In [ ]:
# Configuration for the language model
sampling_config = {
    'temperature':0.1,
    'top_p':0.90,
    'top_k':50,
    'max_tokens':10000,
    'repetition_penalty':1.1,
}

sampling_params = SamplingParams(
                                  **sampling_config,
                                  )

model_config = {
                'model':"TheBloke/Mistral-7B-Instruct-v0.2-AWQ",
                'quantization':"awq",
                'dtype':"half",
                'max_seq_len_to_capture':15000,
                'max_model_len':15000,
                'gpu_memory_utilization': 0.9,
}

In [ ]:
llm = LLM(
            **model_config,
          )

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/904 [00:00<?, ?B/s]

WARNING 06-06 11:08:56 config.py:213] awq quantization is not fully optimized yet. The speed can be slower than non-quantized models.
INFO 06-06 11:08:56 llm_engine.py:161] Initializing an LLM engine (v0.4.3) with config: model='TheBloke/Mistral-7B-Instruct-v0.2-AWQ', speculative_config=None, tokenizer='TheBloke/Mistral-7B-Instruct-v0.2-AWQ', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, rope_scaling=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.float16, max_seq_len=15000, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, disable_custom_all_reduce=False, quantization=awq, enforce_eager=False, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='outlines'), seed=0, served_model_name=TheBloke/Mistral-7B-Instruct-v0.2-AWQ)


tokenizer_config.json:   0%|          | 0.00/1.46k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

INFO 06-06 11:08:59 selector.py:120] Cannot use FlashAttention-2 backend for Volta and Turing GPUs.
INFO 06-06 11:08:59 selector.py:51] Using XFormers backend.
INFO 06-06 11:09:03 selector.py:120] Cannot use FlashAttention-2 backend for Volta and Turing GPUs.
INFO 06-06 11:09:03 selector.py:51] Using XFormers backend.
INFO 06-06 11:09:03 weight_utils.py:207] Using model weights format ['*.safetensors']


model.safetensors:   0%|          | 0.00/4.15G [00:00<?, ?B/s]

INFO 06-06 11:09:27 weight_utils.py:250] No model.safetensors.index.json found in remote.
INFO 06-06 11:09:29 model_runner.py:146] Loading model weights took 3.8814 GB
INFO 06-06 11:09:37 gpu_executor.py:83] # GPU blocks: 3384, # CPU blocks: 2048
INFO 06-06 11:09:41 model_runner.py:854] Capturing the model for CUDA graphs. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI.
INFO 06-06 11:09:41 model_runner.py:858] CUDA graphs can take additional 1~3 GiB memory per GPU. If you are running out of memory, consider decreasing `gpu_memory_utilization` or enforcing eager mode. You can also reduce the `max_num_seqs` as needed to decrease memory usage.
INFO 06-06 11:09:54 model_runner.py:924] Graph capturing finished in 14 secs.


In [ ]:
diciconario = {"lista": []}

In [ ]:
diciconario["lista"].append("hola")
diciconario

{'lista': ['hola']}

In [ ]:
texto_raw  = """Training considerations for strength 4  3. Physiological Factors Affecting Muscular Strength Muscular strength development is underpinned by a combination of several morphological and neural factors.  However, the mechanisms that improve muscular strength are considered multifactorial and can be influenced by other confounders such as initial strength [9], training status [10], and genetics [11].  The following provides a brief overview of the morphological and neural factors that may combine to affect muscular strength. Understanding these factors before discussing training considerations sets the context for the variety of responses in each of these underpinning factors that culminate to elicit muscular strength improvements.  Although a thorough discussion is beyond the scope of this review, it should be noted that an athlete’s history of muscle contraction (e.g. fatigue, post-activation, temperature, etc.) may influence the expression of muscular strength [12, 13].  3.1 Muscle Hypertrophy and Architecture Evidence indicates that residual effects from previous training phases carry-over into future training phases [14, 15]. Therefore, increasing hypertrophy in an effort to subsequently improve one’s strength has to do with potentiation and residual training effects [16-18]. Thus, it appears that there is a sequence or progression of training that, when followed, elicits the greatest benefits from RT.  Specifically, evidence suggests that an order of first increasing the muscle’s cross-sectional area (CSA) (i.e. hypertrophy) and work capacity (i.e. force production capacity) [17-19], followed by a subsequent phasic progression [20, 21], can produce superior strength-power gains.  Alterations in skeletal muscle hypertrophy can greatly impact a muscle’s ability to produce force and power.  Simple observation offers some evidence as to the importance of larger CSAs in creating greater absolute force production; indeed sports with body weight classes, such as powerlifting and weightlifting, support this observation.  The rationale behind this is that a greater muscle fiber CSA, particularly type II fibers, may alter the force-velocity characteristics of the whole muscle [16, 22].  Previous research indicated that strong relationships (r=0.70) existed between muscle CSA and greater force production [23].  Further literature suggested that muscle CSA increases and muscle architecture alterations may account for approximately 50-60% of the changes in force production following short-term RT [24], albeit with relatively untrained subjects.  Physiologically, muscle CSA increases may improve force production due to an increase in the number of cross-bridge interactions between actin and myosin within the previously- and newly-formed sarcomeres.  Kawakami et al. [25] indicated that muscle fiber pennation angles are greater in hypertrophied muscles than in normal muscles.  Larger pennation angles may increase the number of
Training considerations for strength 5  cross-bridge interactions due to the packing of more muscle fascicles within the area.  Despite some evidence to support the association between muscle hypertrophy and strength, it should be noted that changes in muscle size and strength can vary between individuals. Such variance between muscle hypertrophy and subsequent strength changes could be due to time-course differences between the measured adaptation, subsequent expression during the strength task, methodological issues associated with the determination of hypertrophy (e.g. physiological CSA vs. anatomical CSA; magnetic resonance imaging (MRI) and dual-energy x-ray absorptiometry (DEXA) measurements vs. girth measurements, etc.), or that enhanced strength can be affected by other physiological or neural factors beyond CSA [9].  In summary, increases in muscle CSA set a platform that combines with concomitant or subsequent changes in muscle architecture, fiber type, and other neural factors such as motor unit (MU) recruitment and muscle activation pattern to enhance the ability to increase maximum strength [17, 18, 26].  While a number of factors (e.g. muscle damage, metabolic alterations, tension, etc.) may affect the hypertrophic response, a thorough discussion of training methods is beyond the scope of this review.  For further information, readers are directed to a series of recent systematic reviews and meta-analyses that discuss best training practices for improving muscle hypertrophy [27-31].       3.2 Musculotendinous Stiffness Inherent to force production and the subsequent force expression as a measure of strength is the concept of our tissues expressing spring-like behavior which influences subsequent muscle performance [32].  Indeed, increased tissue stiffness (i.e. the relationship between a given force and the amount of stretch the tissue undergoes [33]) can enhance force transmission.  Therefore, tendon stiffness adaptations [34], as well as the structures within the muscle (e.g. actin, myosin, titin, and connective tissue), can influence muscular strength and associated characteristics such as rate of force development (RFD) [35, 36] and power [34, 37].  However, a commonly overlooked aspect of skeletal muscle force generation and expression of strength using the aforementioned measures is the role of the large protein or viscoelastic spring within the sarcomere, titin [38].  Titin could be responsible for generating passive tension in the sarcomere [39], which may be why recent evidence has suggested greater importance of the role of titin in muscle function [35, 39-41].  However, it should be noted that increased sarcoplasmic calcium may actively increase the stiffness of titin, contributing to the stiffness of the entire sarcomere [40].  Therefore, changes in muscular strength and force transmission may be partially influenced by changes in tissue stiffness within and surrounding the muscle.
Training considerations for strength 6  3.3 Motor Unit Recruitment Henneman et al. [42] indicated that MUs are recruited in a sequenced manner based on their size (smallest to largest).  Thus, a pool of MUs will be recruited based on the magnitude of force and RFD required during a given task.  For example, smaller MUs that include slow-twitch type I fibers will be recruited when smaller force magnitudes and RFD are required, while larger MUs that include fast-twitch type IIa/IIx fibers may only be recruited if higher forces and RFD are required.  The recruitment order may be maintained during slow, graded, isometric [43], and ballistic actions [44, 45].  Although lower thresholds for MU recruitment may occur during ballistic-type movements due to the required RFD, the size principle appears to hold [36, 46].    The type and intent of the activity may directly affect which MUs are recruited and how they adapt [46-49].  For example, distance runners may only recruit low-threshold, slow-fatiguing MUs that contain type I fibers given the moderate forces that are required repeatedly during a race.  Due to the nature of the task, high-threshold MUs that contain type II fibers may only be recruited when MUs that contain type I fibers fatigue and additional force production is needed to sustain the activity.  Thus, while type I MUs may increase force production capability, the maximal strength expressed when using a combination of all MU types may still be relatively low in distance runners because of infrequent recruitment of MUs that contain type II fibers during training.  In contrast, weightlifters frequently perform ballistic tasks (e.g. snatch, clean and jerk, etc.) that require both high force and RFD magnitudes, and thus MUs that contain type II fibers are targeted.  Based on the recruitment order and lower recruitment thresholds, weightlifters likely recruit MUs that contain both type I and type II fibers, allowing both MU types to be trained.  Previous research demonstrated that while the orderly recruitment of MUs existed during both slow ramp and ballistic actions following ballistic-type training, MUs were recruited at lower force thresholds [46].  Regarding strength development, it appears to be beneficial to recruit high-threshold MUs during training.  Moreover, ballistic training methods may promote the recruitment of larger MUs that contain type II fibers at lower thresholds, thus raising the potential for positive strength-power adaptations to occur.    3.4 Rate Coding (Firing Frequency) After specific MUs are recruited, the frequency at which the α-motoneurons discharge action potentials to the MU’s muscle fibers can modify its force production properties.  Research indicated that force magnitude may increase
Training considerations for strength 7  300-1500% when the firing frequency of recruited MUs increases from its minimum to its maximum [50].  Additional research indicated that RFD may be impacted by the firing frequency of MUs due to high initial firing frequencies being linked to increased doublet discharges (i.e. two consecutive MU discharges in ≤5ms interval) [46].  Thus, it may be postulated that the increased firing frequency of MUs that results in greater force magnitudes and RFD may aid strength-power development.  Previous research indicated that 12 weeks of ballistic training may enhance MU firing frequency [46].  Thus, it is possible that other ballistic training methods, such as weightlifting movements [51] and sprinting [52], may enhance MU firing frequency, ultimately benefitting strength-power characteristics.  3.5 Motor Unit Synchronization While some literature indicates that MU synchronization may be more related to RFD than to force production magnitude [53], it is possible that simultaneous activation of ≥2 MUs enhances peak force production by expressing greater RFD over short time periods.  Previous research indicated that six weeks of RT increased MU synchronization [54], while another study indicated that MU synchronization strength was larger in both the dominant and non-dominant hands of weightlifters compared to musicians and untrained individuals [55].  These findings are supported by research that suggested heavy RT may increase MU synchronization and force production [56].  While evidence strongly indicates that changes in muscular strength coincide with traditional RT, literature discussing MU synchronization changes following ballistic-type training is somewhat mixed.  One study noted that MU synchronization did not change following ballistic-type training [46], while other studies indicated that MU synchronization was enhanced during ballistic tasks [46, 57].  Practically speaking, it appears that training strategies that include heavy RT and/or ballistic-type movements may improve MU synchronization.  Although research examining changes in MU synchronization within RT literature associated with gross motor movements is limited, the link between improved neuromuscular activation patterns and subsequent force production cannot be discounted.   3.6 Neuromuscular Inhibition Neuromuscular inhibition refers to a reduction in the neural drive of a given muscle group during voluntary muscle actions that may negatively affect force production due to the neural feedback received from muscle and joint receptors [58].  While the previous neural mechanisms may produce positive strength-power adaptations, a neural
Training considerations for strength 8  mechanism that negatively affects strength-power development may affect potential training adaptations.  Previous research indicated that heavy RT may down-regulate Ib afferent feedback to the spinal motoneuron pool, leading to reductions in neuromuscular inhibition and increased force production [56].  Further research reported an enhanced neural drive from both the spinal and supraspinal levels following RT that simultaneously decreased neuromuscular inhibition [59], increased power output via reciprocal inhibition during complex training [60], downregulated recurrent inhibition following explosive-type training [61], and enhanced RFD [62]. """

In [ ]:
type(texto_raw)

str

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=500,
        chunk_overlap=200,
        separators=["\n\n", ".\n", "\n", ". ", " ", ""]
        # separators=["\n\n", "\.\n", "\n", "(?<=\. )", " ", ""], esto es regex
    )


In [ ]:
texts = text_splitter.split_text(clean_raw_text(texto_raw))

In [ ]:
content_generation_dict = {"texts": texts}

In [ ]:
def subject_identification_prompt_template(user_query, context):
    prompt = f"""
<|system|>
You are a an assistant who has to identify if the context in "Context:" is related to the topic the user is writing about in "User_query:". You ONLY can answer "True" or "False" without spaces punctuation marks or anything.</s>
Example of your response. "Response:True"</s>
Context: {context}</s>
<|user|>
User_query: {user_query}</s>
<|assistant|></s>
Response:
"""
    return prompt


In [ ]:
user_query = "Create for me training content about muscular strenght"
subject_identification_prompt = [subject_identification_prompt_template(user_query, context) for context in texts]

In [ ]:
subject_mask = []
for idx, output in enumerate(llm.generate(subject_identification_prompt, sampling_params)):
  print(f"Index: {idx}")
  subject_mask.append(output.outputs[0].text.strip().split('.')[0])
  print(output.outputs[0].text + "\n\n")

Processed prompts: 100%|██████████| 42/42 [00:09<00:00,  4.47it/s, Generation Speed: 101.32 toks/s]

Index: 0
True


Index: 1
False.
The context discusses the factors influencing muscular strength, not creating training content.


Index: 2
False

The context provided discusses the factors affecting muscular strength but does not provide enough information to create training content directly. It's more focused on the background and scientific aspects rather than practical applications.


Index: 3
True. The context discusses factors influencing muscular strength, which is relevant to creating training content about muscular strength.


Index: 4
False. The context discusses muscle hypertrophy and its relationship to strength through potentiation and residual training effects. It does not directly relate to creating training content about muscular strength.


Index: 5
True. The context discusses the importance of following a specific sequence in resistance training to maximize muscular strength. This aligns with the topic of creating training content about muscular strength.


Index: 6
Tr

In [ ]:
subject_mask

['True',
 'False',
 'False\n\nThe context provided discusses the factors affecting muscular strength but does not provide enough information to create training content directly',
 'True',
 'False',
 'True',
 'True',
 'True',
 'True',
 'True',
 'True',
 'True',
 'False',
 'False',
 'True',
 'False',
 'True',
 'True',
 'True',
 'False',
 'True',
 'True',
 'True',
 'True',
 'False',
 'True',
 'True',
 'True',
 'True',
 'True',
 'False',
 'True',
 'True',
 'False',
 'True',
 'True',
 'True',
 'False',
 'False',
 'Part of the context is related to the topic',
 'True',
 'True']

In [ ]:
subject_mask_bool = [s == "True" for s in subject_mask]
subject_mask_bool = ["True" in subject_mask_elem for subject_mask_elem in subject_mask]

In [ ]:
content_generation_dict['context_to_use_raw'] = [context  for context, boolean in zip(content_generation_dict['texts'], subject_mask_bool) if boolean]

In [ ]:
for idx, text in enumerate(content_generation_dict['context_to_use_raw']):
  print(f"Index: {idx}")
  print(text+"\n\n\n")

Index: 0
Training considerations for strength 4 3. Physiological Factors Affecting Muscular Strength Muscular strength development is underpinned by a combination of several morphological and neural factors. However, the mechanisms that improve muscular strength are considered multifactorial and can be influenced by other confounders such as initial strength [9], training status [10], and genetics [11]



Index: 1
. Although a thorough discussion is beyond the scope of this review, it should be noted that an athlete’s history of muscle contraction (e.g. fatigue, post-activation, temperature, etc.) may influence the expression of muscular strength [12, 13]. 3.1 Muscle Hypertrophy and Architecture Evidence indicates that residual effects from previous training phases carry-over into future training phases [14, 15]



Index: 2
. Thus, it appears that there is a sequence or progression of training that, when followed, elicits the greatest benefits from RT. Specifically, evidence suggests t

In [ ]:
def content_reduction_prompt_template(context):
    prompt = f"""
<|system|>
You are a helful assistant that must reduce de content inside de following "Context:" as much as possible without loss of information</s>
In order to reduce as much as possible, it does not have to be understandable by humans only have to be understandable by you.</s>
Context: {context}</s>
<|assistant|></s>
Content_reduced:
"""
    return prompt

In [ ]:
content_reduction_prompt = [content_reduction_prompt_template(context) for context in content_generation_dict['context_to_use_raw']]

In [ ]:
content_reduced_list = []
for idx, output in enumerate(llm.generate(content_reduction_prompt, sampling_params)):
  generation_result = output.outputs[0].text
  print(f"Index: {idx}")
  print(f"Raw input:\n {content_generation_dict['context_to_use_raw'][idx]}\n\n")
  print(f"Content reduction result:\n {generation_result}\n\n")
  content_reduced_list.append(generation_result)

content_generation_dict['content_reduced'] = content_reduced_list

Processed prompts: 100%|██████████| 29/29 [00:09<00:00,  3.07it/s, Generation Speed: 165.06 toks/s]

Index: 0
Raw input:
 Training considerations for strength 4 3. Physiological Factors Affecting Muscular Strength Muscular strength development is underpinned by a combination of several morphological and neural factors. However, the mechanisms that improve muscular strength are considered multifactorial and can be influenced by other confounders such as initial strength [9], training status [10], and genetics [11]


Content reduction result:
 Muscular strength development influenced by morphological, neural factors, initial strength, training status, genetics.


Index: 1
Raw input:
 . Although a thorough discussion is beyond the scope of this review, it should be noted that an athlete’s history of muscle contraction (e.g. fatigue, post-activation, temperature, etc.) may influence the expression of muscular strength [12, 13]. 3.1 Muscle Hypertrophy and Architecture Evidence indicates that residual effects from previous training phases carry-over into future training phases [14, 15]


Co

In [ ]:
'\n'.join(content_reduced_list)

"Muscular strength development influenced by morphological, neural factors, initial strength, training status, genetics.\n1. Athlete's past muscle state (fatigue, activation, temp) impacts muscular strength [12, 13].\n2. Previous training phases affect muscle hypertrophy and architecture [14, 15].\nRT benefits maximized through CSA & work capacity increase first, then phasic progression. [17-19] CSA growth, [17-19] work capacity enhancement, [20, 21] phasic progression.\nHypertrophy and work capacity affect strength-power gains. Muscle hypertrophy impacts force production capacity. Progressive phases enhance these effects.\nMuscle hypertrophy alters force/power capability. Larger Cross-Sectional Areas (CSAs) enhance absolute force production, evident in weightlifting and powerlifting.\nMuscle cross-sectional area (CSA) correlates strongly (r=0.70) with increased force production, especially for type II fibers, which can modify the muscle's force-velocity properties. [16, 22, 23]\nMuscl

In [ ]:
print(sum(len(s) for s in content_generation_dict['context_to_use_raw']))
print(sum(len(s) for s in content_generation_dict['content_reduced']))

10677
6507


In [ ]:
prompt = "develops for me what the following means, knowing that it has been reduced by you'Device_warning: battery_low -> warning_tone, dim_screen, auto_shutoff.' "

In [ ]:
for idx, output in enumerate(llm.generate(prompt, sampling_params)):
  generation_result = output.outputs[0].text
  print(f"Index: {idx}")
  print(generation_result)

Processed prompts: 100%|██████████| 1/1 [00:03<00:00,  3.18s/it, Generation Speed: 29.88 toks/s]

Index: 0


This means that when the device's battery level is low, the following actions will be taken:

1. A warning tone will be played to alert the user.
2. The screen will be dimmed to conserve battery power.
3. The device will automatically shut off to prevent further battery drain.

These actions are designed to help prolong the battery life and prevent the device from shutting down unexpectedly due to low battery.


In [ ]:
joined_list = ['\n\n'.join(content_generation_dict['content_reduced'])]

content_generation_dict['content_reduce_joined'] = joined_list

In [ ]:
for elem in joined_list:
  print(elem)
  print("#######################")

Muscular strength development influenced by morphological, neural factors, initial strength, training status, genetics.

1. Athlete's past muscle state (fatigue, activation, temp) impacts muscular strength [12, 13].
2. Previous training phases affect muscle hypertrophy and architecture [14, 15].

RT benefits maximized through CSA & work capacity increase first, then phasic progression. [17-19] CSA growth, [17-19] work capacity enhancement, [20, 21] phasic progression.

Hypertrophy and work capacity affect strength-power gains. Muscle hypertrophy impacts force production capacity. Progressive phases enhance these effects.

Muscle hypertrophy alters force/power capability. Larger Cross-Sectional Areas (CSAs) enhance absolute force production, evident in weightlifting and powerlifting.

Muscle cross-sectional area (CSA) correlates strongly (r=0.70) with increased force production, especially for type II fibers, which can modify the muscle's force-velocity properties. [16, 22, 23]

Muscle 

In [ ]:
training_objectives = "Create for me training content about muscular strenght"
pedalogical_objectives = "I need the influencing factors in muscular strenght. I need 2 excersises in order to test my knowledge in these factors"

In [ ]:
def content_creation_prompt_template(training_objectives, pedalogical_objectives, context):
    prompt = f"""
<|system|>
You are a an assistant who has to generate training contect based on "User_context_for_content_creation:". You have to take into account "User_training_objectives" and "User_pedalogical_objectives".</s>
The generated content generated in "Generated_content:" have to be in markdown format.</s>
<|user|>
User_training_objectives: {training_objectives}</s>
User_pedalogical_objectives: {pedalogical_objectives}</s>
User_context_for_content_creation: {context}</s>
<|assistant|></s>
Generated_content:
"""
    return prompt

In [ ]:
content_creation_prompt = [content_creation_prompt_template(training_objectives, pedalogical_objectives, context) for context in content_generation_dict['content_reduce_joined']]

In [ ]:
for idx, output in enumerate(llm.generate(content_creation_prompt, sampling_params)):
  print(f"Index: {idx}")
  subject_mask.append(output.outputs[0].text.strip().split('.')[0])
  print(output.outputs[0].text + "\n\n")

Processed prompts: 100%|██████████| 1/1 [00:30<00:00, 30.28s/it, Generation Speed: 27.19 toks/s]

Index: 0

## Factors Influencing Muscular Strength

Muscular strength development is influenced by various factors, including morphological, neural, initial strength, training status, and genetics. Let's explore some of these factors in detail.

### Morphological Factors

An athlete's past muscle state, such as fatigue, activation, and temperature, impacts their muscular strength [12, 13]. Additionally, previous training phases affect muscle hypertrophy and architecture [14, 15].

#### Cross-Sectional Area (CSA) Expansion

Muscle hypertrophy and cross-sectional area (CSA) expansion play crucial roles in muscular strength development. The larger the muscle CSA, the more actin-myosin interactions occur, leading to increased force production [16, 17, 18].

#### Tissue Stiffness

Tissue stiffness, particularly tendon stiffness adaptations and muscle structure modifications, influences muscular strength through force transmission [19, 20, 21].

### Neural Factors

Neural factors, such as mo

In [ ]:
training_objectives = "Create for me training content about muscular strenght"
pedalogical_objectives = "I need the influencing factors in muscular strenght. I need excersises in order to test my knowledge in these factors"

In [ ]:
def content_creation_prompt_template(training_objectives, pedalogical_objectives, context):
    prompt = f"""
You are an expert assistant tasked with generating educational content and a quiz based on the user's inputs.

### User Inputs:
- **Training Objectives:** {training_objectives}
- **Pedagogical Objectives:** {pedalogical_objectives}
- **Context for Content Creation:** {context}

### Instructions:
1. **Content Creation:**
   - Provide a structured, comprehensive explanation of the topic(s) considering the training and pedagogical objectives.
   - Use markdown format with clear headings, subheadings, bullet points, and examples.
2. **Exercise Creation:**
   - Develop exercises to assess understanding of the generated content considering the pedalogical objectives.
   - Use various types of questions (e.g., multiple-choice, true/false, short answer).
   - **Vary the correct answers among the options (e.g., not always option A).**
   - Provide the correct answers


### Generated Content:
"""
    return prompt

In [ ]:
content_creation_prompt = [content_creation_prompt_template(training_objectives, pedalogical_objectives, context) for context in content_generation_dict['content_reduce_joined']]

In [ ]:
for idx, output in enumerate(llm.generate(content_creation_prompt, sampling_params)):
  print(f"Index: {idx}")
  subject_mask.append(output.outputs[0].text.strip().split('.')[0])
  print(output.outputs[0].text + "\n\n")


Processed prompts: 100%|██████████| 2/2 [00:28<00:00, 14.10s/it, Generation Speed: 55.03 toks/s]

Index: 0

## Factors Influencing Muscular Strength

Muscular strength development is influenced by various factors, including morphological, neural, initial strength, training status, and genetics. Let's explore some of these factors in detail.

### Morphological Factors

An athlete's past muscle state, such as fatigue, activation, and temperature, impacts their muscular strength. Additionally, previous training phases can affect muscle hypertrophy and architecture.

#### Cross-Sectional Area (CSA)

Muscle cross-sectional area (CSA) correlates strongly with increased force production, especially for type II fibers. Up to 50-60% of force production enhancements following short-term resistance training can be attributed to muscle CSA growth and architecture modifications.

#### Tissue Stiffness

Tissue stiffness enhances force transmission, influencing muscular strength through tendon stiffness adaptations and muscle structure modifications. This affects RFD and power.

### Neural Factor

In [ ]:
texto_raw = """WARNING! This product contains chemicals known to
the State of California to cause cancer, birth defects, or
other reproductive harm. For more information, please
call 1-800-SAMSUNG (726-7864).
Note: Water-resistant and dustproof based on IP68
rating, which tests submersion up to 5.0 feet for up to
30 minutes.
Intellectual Property
All Intellectual Property, as defined below, owned by
or which is otherwise the property of Samsung or its
respective suppliers relating to the SAMSUNG Phone,
including but not limited to, accessories, parts, or
software relating there to (the “Phone System”), is
proprietary to Samsung and protected under federal
laws, state laws, and international treaty provisions.
Intellectual Property includes, but is not limited to,
inventions (patentable or unpatentable), patents, trade
secrets, copyrights, software, computer programs, and
related documentation and other works of authorship.
You may not infringe or otherwise violate the rights
secured by the Intellectual Property. Moreover,
you agree that you will not (and will not attempt to)
modify, prepare derivative works of, reverse engineer,
decompile, disassemble, or otherwise attempt to
create source code from the software. No title to or
ownership in the Intellectual Property is transferred to
you. All applicable rights of the Intellectual Property
shall remain with SAMSUNG and its suppliers.
Open Source Software
Some software components of this product, including
but not limited to ‘PowerTOP’ and ‘e2fsprogs’,
incorporate source code covered under GNU General
Public License (GPL), GNU Lesser General Public
License (LGPL), OpenSSL License, BSD License and
other open source licenses. To obtain the source code
covered under the open source licenses, please visit:
opensource.samsung.com
Legal Notices
VZW_G930V_EN_UM_TN_PB1_021916_R3
Legal Notices ii
Disclaimer of Warranties;
Exclusion of Liability
EXCEPT AS SET FORTH IN THE EXPRESS
WARRANTY CONTAINED ON THE WARRANTY
PAGE ENCLOSED WITH THE PRODUCT, THE
PURCHASER TAKES THE PRODUCT “AS IS”, AND
SAMSUNG MAKES NO EXPRESS OR IMPLIED
WARRANTY OF ANY KIND WHATSOEVER WITH
RESPECT TO THE PRODUCT, INCLUDING BUT
NOT LIMITED TO THE MERCHANTABILITY OF THE
PRODUCT OR ITS FITNESS FOR ANY PARTICULAR
PURPOSE OR USE; THE DESIGN, CONDITION OR
QUALITY OF THE PRODUCT; THE PERFORMANCE
OF THE PRODUCT; THE WORKMANSHIP OF THE
PRODUCT OR THE COMPONENTS CONTAINED
THEREIN; OR COMPLIANCE OF THE PRODUCT
WITH THE REQUIREMENTS OF ANY LAW, RULE,
SPECIFICATION OR CONTRACT PERTAINING
THERETO. NOTHING CONTAINED IN THE
INSTRUCTION MANUAL SHALL BE CONSTRUED
TO CREATE AN EXPRESS OR IMPLIED WARRANTY
OF ANY KIND WHATSOEVER WITH RESPECT TO
THE PRODUCT. IN ADDITION, SAMSUNG SHALL
NOT BE LIABLE FOR ANY DAMAGES OF ANY KIND
RESULTING FROM THE PURCHASE OR USE OF THE
PRODUCT OR ARISING FROM THE BREACH OF THE
EXPRESS WARRANTY, INCLUDING INCIDENTAL,
SPECIAL OR CONSEQUENTIAL DAMAGES, OR LOSS
OF ANTICIPATED PROFITS OR BENEFITS.
Modification of Software
SAMSUNG IS NOT LIABLE FOR PERFORMANCE
ISSUES OR INCOMPATIBILITIES CAUSED BY YOUR
EDITING OF REGISTRY SETTINGS, OR YOUR
MODIFICATION OF OPERATING SYSTEM SOFTWARE.
USING CUSTOM OPERATING SYSTEM SOFTWARE
MAY CAUSE YOUR DEVICE AND APPLICATIONS
TO WORK IMPROPERLY. YOUR CARRIER MAY
NOT PERMIT USERS TO DOWNLOAD CERTAIN
SOFTWARE, SUCH AS CUSTOM OS.
IF YOUR CARRIER PROHIBITS THIS, IF YOU
ATTEMPT TO DOWNLOAD SOFTWARE ONTO THE
DEVICE WITHOUT AUTHORIZATION, YOU WILL
BE NOTIFIED ON THE SCREEN THAT THE PHONE
WILL NO LONGER OPERATE AND BE INSTRUCTED
Legal Notices iii
TO POWER DOWN THE DEVICE. YOU MUST THEN
CONTACT YOUR CARRIER TO RESTORE THE DEVICE
TO THE CARRIER AUTHORIZED SETTINGS.
Samsung KNOX
Samsung KNOX™ is Samsung’s security platform and
is a mark for a Samsung device tested for security
with enterprise use in mind. Additional licensing fee
may be required. For more information about KNOX,
please refer to: samsung.com/us/knox [030115]
Samsung Electronics America
(SEA), Inc.
Address:
85 Challenger Road
Ridgefield Park,
New Jersey 07660
Phone:
1-800-SAMSUNG (726-7864)
Internet Address:
samsung.com
©2016 Samsung Electronics America, Inc. Samsung,
Samsung Galaxy, Multi Window, S Health, S Voice,
Samsung Milk Music, and Smart Switch are all
trademarks of Samsung Electronics Co., Ltd.
Note: Screen images are simulated. Appearance of
device may vary.
Do you have questions about your
Samsung Mobile Device?
For 24 hour information and assistance, we offer a
new FAQ/ARS System (Automated Response System)
at: samsung.com/us/support
The actual available capacity of the internal memory is
less than the specified capacity because the operating
system and default applications occupy part of the
memory. The available capacity may change when you
upgrade the device.
Legal Notices iv
Nuance®, VSuite™, T9® Text Input, and the Nuance logo
are trademarks or registered trademarks of Nuance
Communications, Inc., or its affiliates in the United
States and/or other countries.
The Bluetooth® word mark, figure mark (stylized
“B Design”), and combination mark (Bluetooth word
mark and “B Design”) are registered trademarks and
are wholly owned by the Bluetooth SIG.
microSD™ and the microSD logo are Trademarks of the
SD Card Association.
Google, the Google logo, Android, the Android
logo, Google Play, Gmail, Google Mail, Google
Maps, Google Music, Google Now, Chrome, Google
Hangouts, Picasa, YouTube, and other marks are
trademarks of Google Inc.
Wi-Fi is a registered trademark of the Wireless Fidelity
Alliance, Inc.
LTE is a trademark of ETSI.
Amazon, Kindle, and all related logos are trademarks
of Amazon.com, Inc. or its affiliates.
Legal Information
Important legal information can be accessed in writing
on the mobile device or at samsung.com
READ THIS INFORMATION BEFORE USING YOUR
MOBILE DEVICE.
Samsung Limited Warranty - This product is
covered under the applicable Samsung Limited
Warranty INCLUDING ITS DISPUTE RESOLUTION
PROCEDURE and your right to opt out of arbitration
within 30 calendar days of the first consumer
purchase. You may opt out by either sending
an email to optout@sea.samsung.com with the
subject line “Arbitration Opt-Out” or by calling
1-800-SAMSUNG (726-7864). For more detailed
procedures, please refer to the “Dispute Resolution
Procedures and Arbitration and Opt-Out” section of
the Limited Warranty.
Legal Notices v
Full written terms and detailed information about
the warranty and obtaining service are available
on the device at: Settings > About phone >
Legal information > Samsung legal or you may
access the online version of the Health/Safety and
Warranty guide for your device at:
English
samsung.com/us/Legal/Phone-HSGuide
Spanish
samsung.com/us/Legal/Phone-HSGuide-SP
The online version of the End User License
Agreement (EULA) for your device can be found
online at:
samsung.com/us/Legal/SamsungLegal-EULA4
Table of Contents vi
Getting Started...............................................................1
Front View...........................................................................2
Back View...........................................................................4
Assemble Your Device.....................................................6
Set Up Your Device.........................................................11
Set Up Accounts on Your Device.................................14
Know Your Device......................................................16
Home Screen...................................................................17
Status Bar.........................................................................23
Navigation.........................................................................25
Notification Panel ............................................................27
Multi Window....................................................................31
Enter Text..........................................................................34
Emergency Mode............................................................37
Calling.................................................................................39
Phone Screen ..................................................................40
Make and Answer Calls .................................................43
Speed Dial........................................................................49
Emergency Calls .............................................................51
Set Up and Access Voicemail ......................................53
Applications....................................................................55
Apps List...........................................................................56
Google Apps....................................................................60
Verizon...............................................................................63
Additional Apps ...............................................................65
Calculator..........................................................................67
Calendar ...........................................................................68
Camera and Video..........................................................71
Clock .................................................................................79
Table of Contents
Table of Contents vii
Contacts............................................................................83
Email..................................................................................89
Gallery...............................................................................93
Memo ................................................................................96
Message+ .........................................................................98
Messages ......................................................................101
My Files..........................................................................105
S Health .........................................................................107
S Voice...........................................................................109
Settings ..........................................................................110
How to Use Settings....................................................111
Airplane Mode ..............................................................113
Wi-Fi................................................................................114
Bluetooth........................................................................118
Mobile Hotspot and Tethering....................................121
Data Usage ...................................................................125
Mobile Networks...........................................................126
Advanced Calling.........................................................127
More Settings................................................................128
Sounds and Vibration..................................................132
Notifications...................................................................138
Display and Wallpaper................................................139
Advanced Features......................................................144
Storage...........................................................................146
Battery............................................................................149
Applications Settings...................................................150
NFC and Payment........................................................153
Themes ..........................................................................155
Location .........................................................................156
Lock Screen and Security ..........................................158
Privacy and Emergency ..............................................165
Easy Mode.....................................................................168
Accounts........................................................................169
Language and Input Settings.....................................171
Backup and Reset .......................................................174
Date and Time ..............................................................177
Accessibility ..................................................................178
Table of Contents viii
Help ................................................................................182
About Phone .................................................................183
System Updates ...........................................................184
Getting Started
Learn about your mobile device
hardware, assembly procedures,
and how to get started using your
new device.
Getting Started 2 Front View
Front View
Home key/Finger scanner
Back
Power key
Front camera
Proximity sensors Earpiece
Recent
Volume keys
Status light
Getting Started 3 Front View
Front View
• Earpiece: Listen to a call.
• Front camera: Take self-portraits and record
videos of yourself.
• Power key: Press and hold to turn the device on
or off, restart it, or for quick access to Emergency
mode. Press to lock or wake up the screen.
• Back: Return to the previous screen or close a
dialog box, menu, or keyboard.
• Home key/Finger scanner: Return to the Home
screen. Activate the Fingerprint security feature.
• Recent: Tap to display recent apps, or touch and
hold for Multi window.
• Volume keys: Adjust the volume of your device’s
sounds and audio playback.
• Status light: Display a light when the battery is
low, charging, or fully charged, when a notification
arrives, or when recording voice.
• Proximity sensors: Detect the presence of objects
near the device.
Getting Started 4 Back View
Back View
USB charger/
Accessory port
Headset jack
SIM card/Memory card tray
Speaker
Microphone
Microphone
Heart rate sensor
Rear camera Flash
Getting Started 5 Back View
Back View
• SIM card/Memory card tray: Install the SIM card
and an optional memory card (not included) for
storage of files. For more information, see “Install
the SIM Card and a Memory Card” on page 6.
• Flash: Illuminate subjects in low-light environments
when taking a photo or recording video.
• Heart rate sensor: Measure your heart rate using
your fingertip when using the S Health™ app.
• Headset jack: Connect an optional headset
(not included).
• USB charger/Accessory port: Connect the
USB charger (included) and other optional
accessories (not included).
• Microphone: Record audio and detect voice
commands.
• Speaker: Play music and other sounds.
• Rear camera: Take pictures and record videos.
Getting Started 6 Assemble Your Device
Assemble Your Device
This section helps you to assemble your
device.
Install the SIM Card and a
Memory Card
When you subscribe to a cellular network, you are
provided with a plug-in SIM card. During activation,
your SIM card is loaded with your subscription details,
such as your Personal Identification Number (PIN),
available optional services, and other features.
Warning: Please be careful when handling the SIM card
to avoid damaging it. Keep all SIM cards out of reach of
small children.
You can also install an optional microSD™ or
microSDHC™ memory card (not included) to add more
memory space to your device.
Note: To improve performance and reliability with
high-definition video recording and playback, it is
recommended you use a memory card (SD card) that
is both rated as at least a Class 10 and provides a high
read speed.
Getting Started 7 Assemble Your Device
Caution: Exercise care when using the removal tool to
eject the internal SIM card/Memory card tray.
1. Insert the removal tool into the hole on the
SIM card/Memory card tray, and then push until
the tray pops out.
2. Place the SIM card on the tray.
• Make sure that the card’s gold contacts face
down into the device and that the card is
positioned as shown.
3. Place the memory card on the tray with the gold
contacts facing down.
4. Slide the SIM card/Memory card tray back into
the slot.
Important: To prevent water and dust from entering the
device, remove any dust on the O-Ring and ensure that
the SIM card tray is closed tightly.
Caution: If the SIM card is not inserted correctly, the
device cannot detect it. Reorient the card and place it
back into the slot if the SIM is not detected.
Getting Started 8 Assemble Your Device
Maintaining Water and
Dust Resistance
Note: Water-resistant and dustproof based on
IP68 rating, which tests submersion up to
5.0 feet for up to 30 minutes.
This device is rated IP68 using the Ingress Protection
rating system.
Your device has been tested in a controlled
environment and shown to be water and dust resistant
in certain circumstances (meets requirements of
classification IP68 as described by the international
standard IEC 60529 - Degrees of Protection provided
by Enclosures [IP Code]; test conditions: 15-35°C,
86-106 kPa, 5 feet, for 30 minutes). Despite this
classification, your device is not impervious to
water damage in any situation. It is important that all
compartments are closed tightly.
Follow these tips carefully to prevent damage to the
device.
• Whenever your device gets wet, dry it thoroughly
with a clean, soft cloth.
• Do not expose the device to salt water or ionized
water, or to water over 5.0 feet deep for more than
30 minutes. If your device is exposed to fresh
water, dry it thoroughly with a clean, soft cloth. If
the device is exposed to any liquid other than fresh
water, rinse the device with fresh water immediately
and dry it thoroughly with a clean, soft cloth.
Failure to rinse the device in fresh water and dry it
as instructed may cause the device to suffer from
operability or cosmetic issues.
• Do not expose the device to water at high pressure,
such as ocean waves or a waterfall.
• If the device is dropped or receives an impact, the
water and dust resistant features of the device may
be damaged.
• The touchscreen and other features may not work
properly if the device is used in water or in other
liquids.
Getting Started 9 Assemble Your Device
Charge the Battery
Your device is powered by a rechargeable, standard
Li-Ion battery. A USB charger (charging head and
USB cable) is included with the device for charging
the battery from any standard outlet.
The battery comes partially charged. A fully
discharged battery requires up to four hours of charge
time. You can use the device while charging.
Note: It is recommended you fully charge the battery
before using your device for the first time.
Note: For fastest charging times, use the Adaptive
Fast Charging charger that came with your device. This
charger provides faster charging only when connected to
devices that support Adaptive Fast Charging.
Caution: Use only charging devices and batteries
approved by Samsung. Samsung accessories
are designed to maximize battery life. Using other
accessories may invalidate your warranty and may
cause damage.
1. Insert the USB cable into the USB charger/
Accessory port.
2. Connect the USB cable to the charging head, and
then plug the charging head into a standard outlet.
3. Unplug the charging head from the outlet and
remove the USB cable from the device when
charging is complete.
Note: This device is also compatible with wireless
charging devices approved by Samsung, which are
sold separately.
Getting Started 10 Assemble Your Device
When to Charge the Battery
When the battery is low and only a few minutes of
operation remain, the device sounds a warning tone at
regular intervals.
In this condition, your device conserves its remaining
battery power by dimming the screen.
When the battery level becomes too low, the device
automatically turns off.
For a quick check of your battery level:
► View the battery charge indicator located in the
top right corner of your device’s display. A solid
color ( ) indicates a full charge.
You can also choose to display a percentage
value. Viewing a percentage value can provide a
better idea of the remaining charge on the battery.
For more information, see “Battery” on page 149.
Getting Started 11 Set Up Your Device
Set Up Your Device
Turn Your Device On or Off
The instructions below explain how to turn your device
on or off.
Turn the Device On
To turn the device on:
► Press and hold the Power key until the device
turns on.
Turn the Device Off
To turn the device off:
1. Press and hold the Power key until the
Device options menu is displayed.
2. Tap Power off, and confirm if prompted.
Reset the Device
If your device stops responding to taps or finger
gestures and you are unable to turn it off, you can
reset your device.
► Press and hold the Power key and the
Volume down key at the same time for
more than seven seconds.
Getting Started 12 Set Up Your Device
Initial Set Up
The first time you turn your device on, a Welcome
screen is displayed. It guides you through the basics
of setting up your device.
Follow the prompts to choose a default language,
connect to a Wi-Fi® network, set up accounts, choose
location services, learn about your device’s features,
and more.
Note: Voice Assistant provides voice prompts and
descriptions of items on the screen to improve
accessibility. During setup, you can enable or disable
Voice Assistant by pressing the Home key quickly
three times.
Smart Switch
Use Smart Switch™ to transfer contacts, photos, and
other content from your old device.
1. Select Smart Switch during setup.
– or –
Tap Apps > Settings > Backup and reset >
Open Smart Switch.
2. Follow the instructions and select the content to
transfer.
Visit samsung.com/smartswitch for more information
and to learn how to use the USB connector to transfer
content faster.
Getting Started 13 Set Up Your Device
Secure Your Device
Use your device’s screen lock features to secure your
device.
Lock the Device
By default, the device locks automatically when the
screen times out.
To lock the device:
► Press the Power key.
Unlock the Device
The default Screen lock on your device is Swipe.
Note: To choose a more secure screen lock,
see “Set a Screen Lock” on page 158.
To unlock the device:
► Press the Power key or Home key, and then drag
your finger across the screen.
Getting Started 14 Set Up Accounts on Your Device
Set Up Accounts on Your Device
Samsung Account
Create a Samsung™ account for access to Samsung
content and apps on your device. When you turn on
your device for the first time, set up your existing
Samsung account or create a new Samsung account.
– or –
To create a new Samsung account or set up your
Samsung account on your device at any time,
use Accounts settings. For more information,
see “Accounts” on page 169.
► From a Home screen, tap Apps > Settings >
Accounts > Add account > Samsung account.
Google Account
Your new device uses your Google™ Account to
fully utilize its Android™ features, including Gmail™,
Google Hangouts™, and the Google Play™ store.
When you turn on your device for the first time, set
up your existing Google Account or create a new
Google Account.
– or –
To create a new Google Account or set up your
Google Account on your device at any time,
use Accounts settings. For more information,
see “Accounts” on page 169.
► From a Home screen, tap Apps > Settings >
Accounts > Add account > Google.
Getting Started 15 Set Up Accounts on Your Device
Factory Reset Protection
When you sign in to a Google Account on your device,
Factory Reset Protection is turned on. Factory Reset
Protection (FRP) prevents other people from using
your device if it is reset to factory settings without
your permission. For example, if your device is lost
or stolen and a factory data reset is performed, only
someone with your Google Account username and
password can use the device.
You will not be able to access the device after
a factory data reset if you do not have your
Google Account username and password. For more
information, see “Factory Data Reset” on page 175.
Caution: You should remove your Google Account
before shipping your device to Samsung or any other
service provider for service and before resetting the
device to factory settings.
Enable Factory Reset Protection
Adding a Google Account to your device automatically
turns on the Factory Reset Protection (FRP) security
feature. For more information, see google.com.
Disable Factory Reset Protection
To disable FRP, remove all Google Accounts from the
device.
1. From a Home screen, tap Apps > Settings.
2. Tap Accounts > Google.
• If you have more than one Google Account on
your device, tap the Google Account you want
to remove.
3. Tap More > Remove account.
Know Your Device
Learn some of the key features of
your mobile device, including the
screens and icons, as well as how
to navigate the device and enter
text using the keyboard.
Warning: Please note that a touch screen
responds best to a light touch from the pad
of your finger or a capacitive stylus. Using
excessive force or a metallic object when
pressing on the touch screen may damage
the tempered glass surface and void the
warranty.
Know Your Device 17 Home Screen
Notification icons Status icons
Home Screen
Apps list
App folder
Status bar
Home screen indicator
Primary app shortcuts
App shortcuts
Widget
Know Your Device 18 Home Screen
Home Screen Features
• Status icons: Displays information about the
device, such as the network signal strength, battery
level, and the time.
• Status bar: Displays device status icons (right) and
notification icons (left).
• App folder: Groups multiple apps into a folder.
• Apps list: Allows quick access to all of your
applications.
• Primary app shortcuts: Displays these shortcuts
on all Home screens.
• Home screen indicator: Indicates which
Home screen is currently visible.
• App shortcuts: Launches your favorite applications.
• Widget: Runs as a self-contained application
(not a shortcut) on a Home screen.
• Notification icons: Displays new messages and
emails, available software updates, and more.
Know Your Device 19 Home Screen
Customize your Home screens by adding
application shortcuts, widgets, folders,
and custom wallpapers.
To access Home screen manager:
► From a Home screen, pinch the screen.
– or –
Touch and hold an empty space on the screen.
Manage Home Screens
Your device comes with multiple Home screens to
hold app shortcuts and widgets. You can add, remove,
change the order of screens, as well as choose a
different main Home screen.
1. From a Home screen, pinch the screen.
2. Use these options to manage screens:
• Add a screen: Swipe to the end of the
screens, and then tap Add.
• Remove a screen: Touch and hold a screen,
and then drag it to Remove.
• Set a screen as the main Home screen: Tap
Main Home screen at the top of a screen.
This screen is displayed when you press the
Home key.
• Change the order of screens: Touch
and hold a screen, and then drag it to a
new position.
• Flipboard Briefing: Swipe to the left of the
screens to find the Flipboard Briefing screen.
To enable or disable this feature, tap On/Off at
the top of the screen.
Know Your Device 20 Home Screen
Wallpapers
Change the look of the Home and Lock screens with
wallpaper. You can display a favorite picture or choose
from preloaded wallpapers.
1. From a Home screen, pinch the screen.
2. Tap Wallpapers to customize.
For more information, see “Display and Wallpaper” on
page 139.
To access Wallpaper settings:
► From a Home screen, tap Apps >  Settings >
Display and wallpaper > Wallpaper.
Widgets
Widgets are self-contained apps that run on a
Home screen. There are many kinds of widgets,
including links to a specific contact, Internet
bookmarks, Gmail and email accounts, and
many others.
Add a Widget to a Home Screen
You can add a widget to a Home screen.
1. From a Home screen, pinch the screen.
2. Tap Widgets.
3. Touch and hold a widget, drag it to a Home screen,
and release it.
Remove a Widget
You can remove a widget that you added to a Home
screen.
1. From a Home screen, touch and hold a widget to
release it.
2. Drag the widget to Remove and release it.
Note: Removing a widget does not delete the widget, it
just removes the widget from the Home screen.
Know Your Device 21 Home Screen
Themes
Set visual themes for all system screens and icons on
your device.
1. From a Home screen, pinch the screen.
2. Tap Themes to customize.
For more information, see “Themes” on page 155.
To access Theme settings:
► From a Home screen, tap Apps > Settings
> Themes.
Screen Grid
Change the number of icons that can be placed on
each Home screen by changing the Screen grid
dimensions.
1. From a Home screen, pinch the screen.
2. Tap Screen grid.
3. Tap a dimension option (4 x 4, 4 x 5, or 5 x 5) to
preview its appearance. The numbers indicate how
many icons can be placed horizontally by how
many icons can be placed vertically.
4. Tap Apply to apply the selected dimensions.
App Shortcuts
You can use app shortcuts to launch an app from any
Home screen.
Add a Shortcut to a Home Screen
You can add a shortcut to a Home screen.
1. From a Home screen, tap Apps.
2. Touch and hold an app shortcut, drag it to a Home
screen, and then release it.
Know Your Device 22 Home Screen
Remove a Shortcut
You can remove a shortcut that you added to a Home
screen.
1. Touch and hold an app shortcut to release it.
2. Drag the app shortcut to Remove and
release it.
Note: Removing a shortcut does not delete the app, it
just removes the shortcut from the Home screen.
Folders
Place folders on a Home screen to organize items.
Create a Folder
You can create an app folder on a Home screen.
1. From a Home screen, touch and hold an app
shortcut and drag it on top of another app shortcut
until a highlighted box appears.
2. Release the app shortcut to create the folder.
• Enter folder name: Enter a name for the
folder.
• Palette: Change the color.
• Add application: Place more apps in the
folder. Tap apps to select them, and then tap
Add.
3. Tap anywhere outside of the folder to close it.
• To add more apps, touch and hold an app
shortcut and drag it to the folder.
Remove a Folder from a Home Screen
You can remove a folder that you added to a
Home screen.
1. Touch and hold the folder until it detaches.
2. Drag the folder to Remove and release it.
Know Your Device 23 Status Bar
Status Bar
The Status bar at the top of the Home
screen provides device information (such as
network status, battery charge, and time) on
the right side and notification alerts on the
left.
Status Icons
Battery full: Battery is fully charged.
Battery low: Battery is low. The yellow
area turns red as the charge level becomes
critical.
Battery charging: Battery is charging.
Vibrate mode: Vibrate mode is enabled.
Status Icons
Mute mode: Mute mode is enabled.
4G LTE active: The device is connected to a
4G LTE wireless network.
Signal strength: Cellular network signal
strength.
Network not available: No wireless network
is available.
Wi-Fi active: Wi-Fi is active.
Wi-Fi available: A Wi-Fi network is available.
NFC enabled: NFC (Near Field
Communication) is enabled.
Know Your Device 24 Status Bar
Notification Icons
Missed call: A call has been missed.
Call in progress: A call is in progress.
Call on hold: A call is on hold.
Speakerphone: A call is using
speakerphone.
New email: New email has arrived.
New Gmail: New Gmail has arrived.
New voicemail: New voicemail message has
been received.
New message: New text or multimedia
message has arrived.
Notification Icons
File download in progress: Download of a
file or app is in progress.
File upload in progress: File is being
uploaded or sent.
GPS active: Location service (GPS) is active
and acquiring a signal.
App updates available: App updates are
available from the Google Play store.
App updates complete: App installation or
updates are complete.
Know Your Device 25 Navigation
Navigation
Learn the basics for how to navigate through
your device.
Touchscreen Navigation
Use the following motions and gestures on your
touchscreen to navigate the device.
Tap
Lightly tap items to select or launch them.
• Tap the keyboard to enter characters or text.
• Tap an item to select it.
• Tap an app shortcut to launch the application.
Touch and Hold
Touch and hold items on the screen to activate them.
• Touch and hold a widget on a Home screen to
move it.
• Touch and hold a field to display a pop-up menu of
options.
Swipe
Lightly drag your finger vertically or horizontally across
the screen.
• Swipe the screen to unlock the device.
• Swipe the screen to scroll through the
Home screens or menu options.
Know Your Device 26 Navigation
Drag
Touch and hold an item, and then move it to a new
location.
• Drag an app shortcut to add it to a Home screen.
• Drag a widget to place it in a new location on a
Home screen.
Pinch and Spread
Pinch the screen using your thumb and forefinger or
spread by moving your fingers apart.
• Pinch the screen to zoom out while viewing a
picture or a web page.
• Spread your fingers on the screen to zoom in while
viewing a picture or a web page.
Common Options
The device uses basic command options to perform
common actions.
• More: Tap to view more menu options for a screen.
This often allows you to select items to delete or
share.
• Delete: Tap to select items to delete.
• Share: Tap to select items to share, and a sharing
method.
• Edit: Tap to customize the screen.
• On/Off button: Tap to turn the option on ( ) or
off ( ).
• Checkbox: Tap to enable or select. When
enabled, a checkmark is displayed. You can often
tap a checkbox at the top of the screen to select
all.
• Menu: Tap to view available options for the field.
Know Your Device 27 Notification Panel
Notification icons on the Status bar display calendar events, device status, and more. For
details, open the Notification panel.
Notification Panel
Clear notifications
View all
Settings
Notification card
Screen brightness
Quick settings
Quick connect
Notification settings
Know Your Device 28 Notification Panel
Notification Panel Features
• Settings: Display the Settings screen.
• View all: See all available Quick settings.
• Clear notifications: Clear all notifications.
• Notification settings: Enable or disable
notifications from each app. For more information,
see “Notifications” on page 138.
• Notification card: View details about a notification.
• Quick connect: Search for nearby devices to
connect with quickly and easily.
• Screen brightness: Adjust screen brightness.
• Quick settings: Quickly enable or disable a feature.
Know Your Device 29 Notification Panel
View the Notification Panel
You can access the Notification panel from any Home
screen.
1. Drag the Status bar down to display the
Notification panel.
2. Swipe down the list to see notification details.
• To open an item, tap it.
• To clear a single notification, drag the
notification left or right.
• To clear all notifications, tap Clear.
3. Drag upward from the bottom of the screen or tap
Back to close the Notification panel.
Quick Settings
In addition to notifications, the Notification panel also
provides quick access to device functions such as
Wi-Fi, allowing you to quickly turn them on or off.
To view additional Quick settings:
1. Drag the Status bar down to display the
Notification panel.
2. Swipe left or right across the visible icons.
To customize the Quick settings that display:
1. Drag the Status bar down to display the
Notification panel.
2. Tap View all, and then tap Edit.
• Quick settings: Drag the icons to change their
positions. The first ten icons appear on the
Notification panel.
• Quick connect: Tap On/Off to show or hide
this feature on the Notification panel.
3. Tap Done to save.
Know Your Device 30 Notification Panel
Quick Connect
Search for nearby devices automatically to connect to
them quickly and easily.
1. Drag the Status bar down to display the
Notification panel.
2. Tap Quick connect. Your device automatically
scans for compatible devices.
• Scan for nearby devices: Scan for compatible
devices.
• Refresh: Update the list of discovered devices.
• More:
- Settings: Configure device visibility and
view information about Quick connect.
- About Quick connect: View Quick connect
information.
- Help: Read help topics for Quick connect.
3. Tap a device from the list and select an option:
• Share content: Choose a file to share with the
device.
• Share Wi-Fi profile: Share available Wi-Fi
profiles with the device.
To disconnect from a device:
1. Drag the Status bar down to display the
Notification panel.
2. Tap Quick connect.
3. Tap Disconnect next to a paired device.
Know Your Device 31 Multi Window
Multi Window
Multitask by using multiple applications at the same time.
Window controls
Know Your Device 32 Multi Window
Activate Multi Window
Apps that support Multi window display together on
a split screen. You can switch between the apps,
adjust the size of their display on the screen, and copy
information from one app to the other.
Note: Multi window is not available for all apps.
1. From any screen, touch and hold Recent.
2. Tap two apps that support Multi window to open
them in split screen view.
- or -
If you are currently using an app that supports
Multi window, tap another app to add it in split
screen view.
You can also launch recently used apps directly into
Multi window view:
1. Tap Recent.
• Recently used apps that support Multi window
have a Multi window icon in their title bar.
2. Tap Multi window in the title bar to launch an
application in split screen view.
• Other recently used apps that support
Multi window display below the launched app.
3. Tap an app icon to add it to the split screen view.
Note: Only applications that support Multi window have
the Multi window icon.
Know Your Device 33 Multi Window
Window Controls
Select an app window, and then tap
Window controls in the middle of the
window border.
Controls
Switch windows: Switch locations between
Multi window apps.
Drag and drop content: Move items
between the app windows. Drag an item to
share from one window to another. Not all
applications support this feature.
Minimize window: Minimize the selected
app window. This turns the app into a
circular, movable icon that can be opened
by tapping on it.
Maximize window: Open the selected app
window in full screen view.
Controls
Close app: Close the selected app.
Know Your Device 34 Enter Text
Enter Text
Text can be entered using a keyboard or by
speaking.
Use the Samsung Keyboard
Enter text using a QWERTY keyboard.
Options
Predictive text
Know Your Device 35 Enter Text
Special Keys
Symbol: Tap to insert numbers and symbols.
Touch and hold Options for the following:
Google voice typing: Enter text using
Google voice™ typing.
Clipboard: Access the clipboard.
Insert emoji: Insert an emoji at the
cursor position.
Settings: Access keyboard settings.
Note: Available options depend on the app in use. Not
all options are available for all apps.
Predictive Text
Predictive text suggests words that match your text
entries, and can automatically complete common
words. Predictive text can learn your personal writing
style from your contacts, messages, email, and social
networking sites—improving the accuracy of predicting
text entries.
Configure the Samsung Keyboard
To customize the Samsung keyboard:
1. From a Home screen, tap Apps > Settings.
2. Tap Language and input > Samsung keyboard
for options.
Know Your Device 36 Enter Text
Use Google Voice Typing
Instead of typing, enter text by speaking.
Settings Delete text
To enable Google voice typing:
1. Tap Google voice typing on the Samsung
keyboard.
• You may need to touch and hold Options
(left of the Space bar) to find it.
2. Speak your text.
Note: Tap Settings > Languages to download
additional languages for use with Google voice typing.
Configure Google Voice Typing
To customize Google voice typing:
1. From a Home screen, tap Apps > Settings.
2. Tap Language and input > Google voice typing
for options.
Know Your Device 37 Emergency Mode
Emergency Mode
Emergency mode conserves your device’s
remaining power when you are in an
emergency situation.
Emergency mode saves battery power by:
• Changing the color of the screen to grayscale.
• Restricting application usage to only essential
applications and those you select.
• Turning off Mobile data when the screen is off.
• Turning off connectivity features such as Wi-Fi and
Bluetooth®.
Note: When Emergency mode is turned off, the Location
settings may stay the same as they were in Emergency
mode.
Activate Emergency Mode
To activate Emergency mode:
1. Press and hold the Power key to display the
Device options window.
2. Tap Emergency mode > Turn on.
• When accessing for the first time, read the
Terms and conditions. If you accept them,
tap the Checkbox, and then tap Agree.
Confirm when prompted.
Turn Off Emergency Mode
► From the Emergency mode Home screen, tap
More > Turn off Emergency mode.
Know Your Device 38 Emergency Mode
Use Emergency Mode
While in Emergency mode, your device conserves
power. The following apps and features are available
on the Home screen:
• Flashlight: Use the device’s flash as a steady
source of light.
• Emergency alarm: Sound an audible siren.
• Message my location: Send your location
information to your emergency contacts.
• Phone: Launch the call screen.
• Emergency alerts: Receive geographically-targeted
messages.
• Add: Add other available apps.
- Chrome: Browse the internet.
- Maps: Launch Google Maps™.
• Battery percentage: Display estimated battery
charge remaining.
• Estimated usage time remaining: Display
estimated standby time that can be reached based
on current battery charge and usage.
• Emergency call: Dial the emergency telephone
number (for example, 911). This kind of call can be
made even without activated cellular service.
• More:
- Turn off Emergency mode: Disable Emergency
mode and return to standard mode.
- Remove: Choose apps to remove from the
screen.
- Manage emergency contacts: Add emergency
contacts if none have been set in Safety
Assistance.
- Settings: Configure the available settings. Only
a limited number of settings are enabled in
Emergency mode.
Calling
Make telephone calls using a
variety of calling features and
services.
Calling 40 Phone Screen
Use the Phone screen to access the dialer keypad, recent calls, favorites, and contacts.
Phone Screen
More options
Hide the keypad
Video call
Tabs
Keypad
Voice command
Call
Voicemail
Calling 41 Phone Screen
Phone Features
• More options: Access Speed dial and Call settings.
• Hide the keypad: Hide the keypad and display
recent calls.
• Video call: Make video calls.
• Call: Make a call.
• Voice command: Say the number you want to dial.
• Keypad: Dial the number that you want to call.
• Voicemail: Set up and check your voicemail.
For more information, see
“Set Up and Access Voicemail” on page 53.
• Tabs: Display a log of all dialed, received, and
missed calls. Display contacts marked as favorites.
Display your contacts and select a contact to call.
Calling 42 Phone Screen
Access Phone
There are a couple of ways to access Phone.
► From a Home screen, tap Phone.
– or –
From a Home screen, tap Apps > Phone.
Display Your Phone Number
Your phone number can be found in Settings.
► From a Home screen, tap Apps > Settings >
About phone > Status.
• Your phone number is displayed in the
My phone number field.
Call Settings
There are also a couple of ways to access the
Call settings menu.
► From a Home screen, tap Phone > More >
Settings.
– or –
From a Home screen, tap Apps > Settings >
Applications > Phone.
Calling 43 Make and Answer Calls
Make and Answer Calls
Learn how to make or answer a call. Explore
features and functionality associated with
making or answering a call.
Make a Call
You can make a call from a Home screen.
1. From a Home screen, tap Phone.
2. Enter a number on the keypad and tap Dial.
• Tap Keypad if the keypad is not displayed.
Dial a Number from Your Contacts List
You can quickly dial a phone number from your
Contacts list.
1. From a Home screen, tap Apps > Contacts.
– or –
From a Home screen, tap Phone > Contacts.
2. Swipe your finger across a contact to the right to
call the contact.
Dial a Recent Number
All incoming, outgoing, and missed calls are recorded
in the Call log. If the number or caller is listed in your
contacts list, the associated name is also displayed.
1. From a Home screen, tap Phone.
2. Tap Recent. A list of recent calls is displayed.
3. Tap a contact, and then tap Call.
Calling 44 Make and Answer Calls
Video Calls
To activate Video calling:
1. From a Home screen, tap Phone > More >
Settings.
2. Tap Video call to enable.
To make Video calls:
1. From a Home screen, tap Phone.
2. Enter a number and tap Video call.
Note: Not all devices are compatible for Video calls. The
caller has the option to accept the video call or answer
the call as a regular voice call.
To switch from a Phone call to a Video call after
dialing:
► Tap Call type > Video call to request two-way
video calling.
– or –
Tap Call type > Show me to only send video of
yourself to your recipient.
Answer a Call
When a call is received, the phone rings and the
caller’s phone number or name is displayed, if it is
stored in the Contacts list.
► At the incoming call screen:
• Drag Answer to the right to answer the call.
• Drag Reject to the left to reject the call and
send it to your voicemail.
• Drag Reject call with message upward and
select a message.
Calling 45 Make and Answer Calls
If you are using an app, a pop-up screen is displayed
for the incoming call.
► On the incoming call pop-up screen:
• Tap Answer to answer the call.
• Tap Reject to reject the call and send it to
your voicemail.
• Tap Reject call with message and select a
message.
• Tap Expand to exit the app and view the
main incoming call screen.
End a Call
To finish your call:
► Tap End when you are ready to end your call.
Multitask While on a Call
If you exit the call screen and return to a
Home screen, your active call is indicated by
Call on the Status bar.
This lets you continue with your current call and do
something else, such as access the contacts list, view
a recent email, or locate a picture.
To return to the call screen:
► Drag the Status bar down to display the
Notification panel, and then tap Call notification.
To end a call from any application:
► Drag the Status bar down to display the
Notification panel, and then tap End call.
Calling 46 Make and Answer Calls
Options During a Call
Your phone provides several options that you can use
during a call.
Adjust the Call Volume
► Press the Volume keys to increase or decrease
the volume.
– or –
Tap More > Turn on extra volume to quickly add
more volume to the incoming audio.
Listen with the Speaker or
Bluetooth Headset
Listen to the call using the speaker or through a
Bluetooth headset (not included).
► Tap Speaker to hear the caller using the
speaker.
– or –
Tap Bluetooth to hear the caller using a
Bluetooth headset.
Place a New Call While on a Call
If your service plan supports this feature, you can
make another call while a call is in progress.
1. From the active call, tap Add call to dial the
second call.
2. Dial the new number and tap Dial. When the
call is answered:
• Tap Swap to switch between the two calls.
• Tap Merge to hear both callers at once
(multi-conferencing).
Calling 47 Make and Answer Calls
Call Waiting
If your service plan supports this feature, you can
answer an incoming call while you have a call in
progress. You are notified of an incoming call by a call
waiting tone.
To answer a new call while you have a call in progress:
1. Drag Answer in any direction to answer the
new call.
2. Tap an option:
• Put [caller] on hold to place the previous
caller on hold while you answer the new
incoming call.
• Merge with [caller] to add the new caller to
the current call.
• End call with [caller] to end the previous call
and answer the new call.
3. Tap the previous On hold number entry and
select Swap to switch between the two calls.
This places the new caller on hold and activates
the previous call.
Call Log
The numbers of the calls you have dialed, received, or
missed are stored in the Call log.
1. From a Home screen, tap Phone.
2. Tap Recent. A list of recent calls is displayed. If
the caller is in your Contacts list, the caller’s name
is displayed.
Save a Recent Call to Your Contacts List
Use recent call information to create a contact or
update your Contacts list.
1. From a Home screen, tap Phone > Recent.
2. Tap the call that contains the information that you
want to save to your Contacts list.
3. Tap Create contact.
– or –
If you want to replace the number for an existing
contact, tap Update existing.
Calling 48 Make and Answer Calls
Delete a Call from the Call Log
To delete Call log entries:
1. From a Home screen, tap Phone > Recent.
2. Touch and hold the call you want to delete from
the Call log.
3. Tap Delete.
Block a Number
By adding a caller to your Block list, future calls from
this number are sent directly to your voicemail, and
messages are not received.
1. From a Home screen, tap Phone > Recent.
2. Tap the caller you want to add to the Block list.
3. Tap More > Add to block list:
You can also modify the Block list in Settings:
► From a Home screen, tap Phone > More >
Settings > Call blocking > Block list.
Calling 49 Speed Dial
Speed Dial
You can assign a shortcut number to a
contact for speed dialing their default
number.
Create a Speed Dial
Your can assign 100 speed dial numbers.
1. From a Home screen, tap Phone.
• Tap Keypad if the keypad is not displayed.
2. Tap More > Speed dial. The Speed dial screen
displays the numbers 1 through 100. Number 1 is
reserved for Voicemail.
3. Tap an unassigned number ( Add contact).
4. Tap a contact to assign to the number.
• The selected contact is displayed in the
Speed dial number box.
Use Speed Dial to Make a Call
You can make a call using Speed dial.
1. From a Home screen, tap Phone.
• Tap Keypad if the keypad is not displayed.
2. Touch and hold the Speed dial number.
• If the Speed dial number is more than one
digit long, enter the first digits, and then hold
the last digit.
Calling 50 Speed Dial
Remove a Speed Dial Number
Assignment
You can remove an assigned Speed dial number.
1. From a Home screen, tap Phone.
2. Tap More > Speed dial.
3. Find the contact to remove from Speed dial.
4. Tap Remove.
Calling 51 Emergency Calls
Emergency Calls
You can dial the emergency telephone
number in your region regardless of the
phone’s cellular service status.
Make an Emergency Call
You can dial the emergency telephone number
from your phone whether or not cellular service has
been activated. If your phone is not activated, you can
only make an emergency call.
1. From a Home screen, tap Phone.
2. Enter the emergency telephone number (911 in
North America) and tap Dial.
3. Complete your call. During this type of call, you
have access to most in-call features.
Call Emergency Services from a
Locked Screen
The emergency telephone number can be dialed
even if the phone is locked, allowing anyone to use
your phone to call for help in an emergency. When
accessed from a locked screen, only the emergency
calling feature is accessible to the caller. The rest of
the phone remains secured.
Calling 52 Emergency Calls
Swipe Lock Screen
If the Lock screen is set to Swipe, you can access the
call screen in a couple of ways.
1. Drag Phone upward to open the call screen.
- or -
Swipe across the screen to unlock it, and then tap
Phone.
2. Enter the emergency telephone number (911 in
North America) and tap Dial.
3. Complete your call. During this type of call, you
have access to most in-call features.
Secure Lock Screens
You can make calls even if your phone has a secure
lock screen.
Note: The first step differs depending on the device and
Lock screen settings.
1. Drag Phone upward to open the call screen.
2. Tap Emergency call to display the Emergency
dialer.
3. Enter the emergency telephone number (911 in
North America) and tap Dial.
4. Complete your call. During this type of call, you
have access to most in-call features.
Calling 53 Set Up and Access Voicemail
Set Up and Access Voicemail
Set up and configure your voicemail.
Set Up Your Voicemail
You can access your voicemail from a Home screen.
1. From the Home screen, tap Apps >
Voice mail.
2. Follow the tutorial to create a password, record a
greeting, and record your name.
Note: These steps may be different depending on your
network.
Access Your Voicemail
You can access your voicemail from a Home screen.
1. From the Home screen, tap Apps >
Voice mail.
• You may be prompted to enter a password.
2. Follow the voice prompts from the voicemail center.
Calling 54 Set Up and Access Voicemail
Access Your Voicemail from
Another Phone
You can access your voicemail from another phone.
1. From another phone, dial your wireless phone
number. Wait until you hear your voicemail
greeting.
2. Press the asterisk or star (*) key on the phone
you are using.
3. Enter your password and you are connected to
your voicemail"""

In [ ]:
type(texto_raw)

str

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=500,
        chunk_overlap=200,
        separators=["\n\n", ".\n", "\n", ". ", " ", ""]
        # separators=["\n\n", "\.\n", "\n", "(?<=\. )", " ", ""], esto es regex
    )


In [ ]:
texts = text_splitter.split_text(clean_raw_text(texto_raw))

In [ ]:
content_generation_dict = {"texts": texts}

In [ ]:
def subject_identification_prompt_template(user_query, context):
    prompt = f"""
<|system|>
You are a an assistant who has to identify if the context in "Context:" is related to the topic the user is writing about in "User_query:". You ONLY can answer "True" or "False" without spaces punctuation marks or anything.</s>
Example of your response. "Response:True"</s>
Context: {context}</s>
<|user|>
User_query: {user_query}</s>
<|assistant|></s>
Response:
"""
    return prompt


In [ ]:
user_query = "Create for me training content about how manage the windows in the smartphone and what options I have to write text with the phone"
subject_identification_prompt = [subject_identification_prompt_template(user_query, context) for context in texts]

In [ ]:
subject_mask = []
for idx, output in enumerate(llm.generate(subject_identification_prompt, sampling_params)):
  print(f"Index: {idx}")
  subject_mask.append(output.outputs[0].text.strip().split('.')[0])
  print(output.outputs[0].text + "\n\n")


Processed prompts: 100%|██████████| 158/158 [01:02<00:00,  2.53it/s, Generation Speed: 90.86 toks/s]


Index: 0
False


Index: 1
False.


Index: 2
False.
The context is about Intellectual Property and its protection, while the user query is about managing windows in a smartphone and text input options.


Index: 3
False


Index: 4
False.


Index: 5
False.


Index: 6
False.
The context is about a product warranty, which is not related to managing windows or writing text on a smartphone.


Index: 7
False.
The context is about product quality, performance, workmanship, compliance, and disclaimers. The user query is about managing windows and text input on a smartphone.


Index: 8
False


Index: 9
False.
The context is about software modification and liability issues, while the user query is about managing windows and text input on a smartphone.


Index: 10
False


Index: 11
False.
The context is about Samsung KNOX, a security platform for Samsung devices. The user query is about managing windows and text input on a smartphone.


Index: 12
True (for the context being related to the topic of 

In [ ]:
subject_mask

['False',
 'False',
 'False',
 'False',
 'False',
 'False',
 'False',
 'False',
 'False',
 'False',
 'False',
 'False',
 'True (for the context being related to the topic of smartphones and their features, including Multi Window)\nFalse (for the context not directly addressing the specific user query about managing text input)',
 'False',
 'False',
 'False',
 'False',
 'False',
 'False',
 'False',
 'True (for the context)\nFalse (for the user query)',
 'True\n\nContext is related to the topic in User_query',
 'True\n\nContext: Contacts',
 'True\n\nThe context mentions "Display and Wallpaper" and "Settings" which are related to managing windows and display settings on a smartphone',
 'True (for Context: Display and Wallpaper, Advanced Features, Language and Input Settings)\nFalse (for all other contexts)',
 'True\n\nContext: Language and Input Settings',
 'False\n\nContext is about the front view and features of a smartphone, while the user query is about managing windows and writing te

In [ ]:
#subject_mask_bool = [s == "True" for s in subject_mask]

subject_mask_bool = ["True" in subject_mask_elem for subject_mask_elem in subject_mask]
subject_mask_bool

[False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 True,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 True,
 True,
 True,
 True,
 True,
 True,
 False,
 True,
 True,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 True,
 True,
 False,
 False,
 True,
 False,
 False,
 True,
 False,
 False,
 True,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 True,
 True,
 True,
 True,
 True,
 True,
 False,
 False,
 True,
 False,
 True,
 False,
 False,
 False,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 True,
 True,
 False,
 False,
 False,
 False,
 False,
 False,
 Fal

In [ ]:
content_generation_dict['context_to_use_raw'] = [context  for context, boolean in zip(content_generation_dict['texts'], subject_mask_bool) if boolean]

In [ ]:
for idx, text in enumerate(content_generation_dict['context_to_use_raw']):
  print(f"Index: {idx}")
  print(text+"\n\n\n")

Index: 0
.
Address:
85 Challenger Road
Ridgefield Park,
New Jersey 07660
Phone:
1-800-SAMSUNG (726-7864)
Internet Address:
samsung.com
©2016 Samsung Electronics America, Inc. Samsung, 
Samsung Galaxy, Multi Window, S Health, S Voice, 
Samsung Milk Music, and Smart Switch are all 
trademarks of Samsung Electronics Co., Ltd.
Note: Screen images are simulated. Appearance of 
device may vary



Index: 1
English
samsung.com/us/Legal/Phone-HSGuide
Spanish
samsung.com/us/Legal/Phone-HSGuide-SP
The online version of the End User License 
Agreement (EULA) for your device can be found 
online at:
samsung.com/us/Legal/SamsungLegal-EULA4
Table of Contents vi
Getting Started...1
Front View...2
Back View...4
Assemble Your Device...6
Set Up Your Device...11
Set Up Accounts on Your Device...14
Know Your Device...16
Home Screen...17
Status Bar...23
Navigation...25
Notification Panel ...27
Multi Window...31



Index: 2
Assemble Your Device...6
Set Up Your Device...11
Set Up Accounts on Your Device...14


In [ ]:
def content_reduction_prompt_template(context):
    prompt = f"""
<|system|>
You are a helful assistant that must reduce de content inside de following "Context:" as much as possible without loss of information</s>
In order to reduce as much as possible, it does not have to be understandable by humans only have to be understandable by you.</s>
Context: {context}</s>
<|assistant|></s>
Content_reduced:
"""
    return prompt

In [ ]:
content_reduction_prompt = [content_reduction_prompt_template(context) for context in content_generation_dict['context_to_use_raw']]

In [ ]:
content_reduced_list = []
for idx, output in enumerate(llm.generate(content_reduction_prompt, sampling_params)):
  generation_result = output.outputs[0].text
  print(f"Index: {idx}")
  print(f"Raw input:\n {content_generation_dict['context_to_use_raw'][idx]}\n\n")
  print(f"Content reduction result:\n {generation_result}\n\n")
  content_reduced_list.append(generation_result)

content_generation_dict['content_reduced'] = content_reduced_list


Processed prompts: 100%|██████████| 36/36 [00:21<00:00,  1.65it/s, Generation Speed: 121.79 toks/s]

Index: 0
Raw input:
 .
Address:
85 Challenger Road
Ridgefield Park,
New Jersey 07660
Phone:
1-800-SAMSUNG (726-7864)
Internet Address:
samsung.com
©2016 Samsung Electronics America, Inc. Samsung, 
Samsung Galaxy, Multi Window, S Health, S Voice, 
Samsung Milk Music, and Smart Switch are all 
trademarks of Samsung Electronics Co., Ltd.
Note: Screen images are simulated. Appearance of 
device may vary


Content reduction result:
 Address: Ridgefield Park, NJ, 07660, USA
Phone: 1-800-726-7864
Website: samsung.com
Trademarks: Samsung, Samsung Galaxy, Multi Window, S Health, S Voice, Samsung Milk Music, Smart Switch (all trademarks of Samsung Electronics Co., Ltd.)
Simulated screen images. Device appearance may differ. ©SEA, Samsung Electronics America, Inc. 2016.


Index: 1
Raw input:
 English
samsung.com/us/Legal/Phone-HSGuide
Spanish
samsung.com/us/Legal/Phone-HSGuide-SP
The online version of the End User License 
Agreement (EULA) for your device can be found 
online at:
samsung.com/us/L

In [ ]:
'\n'.join(content_reduced_list)

'Address: Ridgefield Park, NJ, 07660, USA\nPhone: 1-800-726-7864\nWebsite: samsung.com\nTrademarks: Samsung, Samsung Galaxy, Multi Window, S Health, S Voice, Samsung Milk Music, Smart Switch (all trademarks of Samsung Electronics Co., Ltd.)\nSimulated screen images. Device appearance may differ. ©SEA, Samsung Electronics America, Inc. 2016.\nEnglish EULA: samsung.com/us/Legal/SamsungLegal-EULA4\nSpanish EULA: samsung.com/us/Legal/SamsungLegal-EULA4-SP\nOnline location for Samsung device\'s EULA.\nTOC: Getting Started, Front View, Back View, Assemble, Set Up, Accounts, Know Your Device, Home Screen, Status Bar, Navigation, Notification Panel, Multi Window.\nDevice_setup: 6 (Assemble, Set Up)\nAccounts: 14\nDevice_knowledge: 16 (Know Your Device)\nHome_screen: 17\nStatus_bar: 23\nNavigation: 25\nNotification_panel: 27\nMulti_window: 31\nText_input: 34\nEmergency_mode: 37\nCalling: 43 (Make, Answer)\nScreen: 40 (Phone)\nCall_functions: 49-53 (Speed Dial, Emergency, Voicemail)\nApps: 55\nA

In [ ]:
print(sum(len(s) for s in content_generation_dict['context_to_use_raw']))
print(sum(len(s) for s in content_generation_dict['content_reduced']))

15966
8791


In [ ]:
for elem in content_generation_dict['content_reduced']:
  print(elem)
  print("############################")

Address: Ridgefield Park, NJ, 07660, USA
Phone: 1-800-726-7864
Website: samsung.com
Trademarks: Samsung, Samsung Galaxy, Multi Window, S Health, S Voice, Samsung Milk Music, Smart Switch (all trademarks of Samsung Electronics Co., Ltd.)
Simulated screen images. Device appearance may differ. ©SEA, Samsung Electronics America, Inc. 2016.
############################
English EULA: samsung.com/us/Legal/SamsungLegal-EULA4
Spanish EULA: samsung.com/us/Legal/SamsungLegal-EULA4-SP
Online location for Samsung device's EULA.
TOC: Getting Started, Front View, Back View, Assemble, Set Up, Accounts, Know Your Device, Home Screen, Status Bar, Navigation, Notification Panel, Multi Window.
############################
Device_setup: 6 (Assemble, Set Up)
Accounts: 14
Device_knowledge: 16 (Know Your Device)
Home_screen: 17
Status_bar: 23
Navigation: 25
Notification_panel: 27
Multi_window: 31
Text_input: 34
Emergency_mode: 37
Calling: 43 (Make, Answer)
Screen: 40 (Phone)
Call_functions: 49-53 (Speed Dial,

In [ ]:
prompt = "develops for me what the following means, knowing that it has been reduced by you'Device_warning: battery_low -> warning_tone, dim_screen, auto_shutoff.' "

In [ ]:
for idx, output in enumerate(llm.generate(prompt, sampling_params)):
  generation_result = output.outputs[0].text
  print(f"Index: {idx}")
  print(generation_result)


Processed prompts: 100%|██████████| 1/1 [00:06<00:00,  6.54s/it, Generation Speed: 26.34 toks/s]

Index: 0


This is a rule in the context of an IoT (Internet of Things) system, where a device sends a message (in this case, a 'Device_warning' event) with a payload (in this case, 'battery_low') that triggers a specific action.

The rule states that when the device sends a 'Device_warning' event with the payload 'battery_low', the following actions should be taken:

1. Play a warning tone.
2. Dim the screen.
3. Auto-shutoff.

So, in simple terms, when the device's battery level gets low, it sends a warning message to the system, and the system responds by playing a warning tone, dimming the screen, and automatically shutting off to save power.


In [ ]:
joined_list = ['\n\n'.join(content_generation_dict['content_reduced'])]

content_generation_dict['content_reduce_joined'] = joined_list

In [ ]:
for elem in joined_list:
  print(elem)
  print("#######################")

Address: Ridgefield Park, NJ, 07660, USA
Phone: 1-800-726-7864
Website: samsung.com
Trademarks: Samsung, Samsung Galaxy, Multi Window, S Health, S Voice, Samsung Milk Music, Smart Switch (all trademarks of Samsung Electronics Co., Ltd.)
Simulated screen images. Device appearance may differ. ©SEA, Samsung Electronics America, Inc. 2016.

English EULA: samsung.com/us/Legal/SamsungLegal-EULA4
Spanish EULA: samsung.com/us/Legal/SamsungLegal-EULA4-SP
Online location for Samsung device's EULA.
TOC: Getting Started, Front View, Back View, Assemble, Set Up, Accounts, Know Your Device, Home Screen, Status Bar, Navigation, Notification Panel, Multi Window.

Device_setup: 6 (Assemble, Set Up)
Accounts: 14
Device_knowledge: 16 (Know Your Device)
Home_screen: 17
Status_bar: 23
Navigation: 25
Notification_panel: 27
Multi_window: 31
Text_input: 34
Emergency_mode: 37
Calling: 43 (Make, Answer)
Screen: 40 (Phone)
Call_functions: 49-53 (Speed Dial, Emergency, Voicemail)
Apps: 55
App_list: 56
Google_apps

In [ ]:
training_objectives = "I need my workers to know all the things about how manage the windows in the smartphone and what options I have to write text with the phone"
pedalogical_objectives = "I need enumerated steps for doing the thins and the options that i have and I need 2 excersises in order to test the abbility of my workers after training over the training content that you generated."

In [ ]:
def content_creation_prompt_template(training_objectives, pedalogical_objectives, context):
    prompt = f"""
<|system|>
You are a an assistant who has to generate training contect based on "User_context_for_content_creation:". You have to take into account "User_training_objectives" and "User_pedalogical_objectives".</s>
The generated content generated in "Generated_content:" have to be in markdown format.</s>
<|user|>
User_training_objectives: {training_objectives}</s>
User_pedalogical_objectives: {pedalogical_objectives}</s>
User_context_for_content_creation: {context}</s>
<|assistant|></s>
Generated_content:
"""
    return prompt

In [ ]:
content_creation_prompt = [content_creation_prompt_template(training_objectives, pedalogical_objectives, context) for context in content_generation_dict['content_reduce_joined']]

In [ ]:
for idx, output in enumerate(llm.generate(content_creation_prompt, sampling_params)):
  print(f"Index: {idx}")
  subject_mask.append(output.outputs[0].text.strip().split('.')[0])
  print(output.outputs[0].text + "\n\n")


Processed prompts: 100%|██████████| 1/1 [00:41<00:00, 41.27s/it, Generation Speed: 22.88 toks/s]

Index: 0

# Managing Windows and Text Input on Samsung Galaxy Devices

## Getting Started

Before diving into the specifics of managing windows and text input on your Samsung Galaxy device, let's familiarize ourselves with the layout and key features.

### Front View

![Front View](front_view.png)

The front view shows the Home screen, which is where you'll find your most frequently used apps and widgets. The Home button, located at the bottom center, takes you back to the Home screen whenever pressed.

### Back View

![Back View](back_view.png)

The back view reveals the camera lens and the flashlight. To take a picture, open the Camera app and press the large circle at the bottom center. To turn on the flashlight, swipe up from the Home screen and tap the Flashlight icon.

## Setting Up Your Samsung Galaxy Device

Now that we've taken a look around, let's set up your device.

### Accounts

To get started, you'll need to create or sign in to your accounts. This includes your Google Ac